In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import csv
import os
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from numpy import array
from sklearn.model_selection import train_test_split

In [ ]:
r_offs = []
r_qlngth = []
r_ptch = []
train_data = []
i = 0

for directory, subdirectories, files in os.walk('data'):
    for file in files:
        with open('data/' + file, 'r') as f: 
	        reader = csv.reader(f)
	        stream_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]

        offsb = stream_list[-1][0]
        qlngthb = max(element[1] for element in stream_list)
        #ptchb = 127.0

        for row in stream_list:
            r_offs.append(row[0]/offsb)
            r_qlngth.append(row[1]/qlngthb)
            r_ptch.append(row[2])
            #train_data.append([r_offs[i], r_qlngth[i]])
            i += 1

In [ ]:
y = np_utils.to_categorical(r_ptch)

#i = 0
#for row in y:
#    train_data[i].append(y[i])
#    i += 1

In [ ]:
dtlngth=np.shape(r_offs)[0]
seq_length = 100#100 groups of 3

dataX_o = []
dataY_o = []
dataX_q = []
dataY_q = []
dataX_p = []
dataY_p = []

for i in range(0, dtlngth - seq_length, 1):
	seq_in_o = r_offs[i:i + seq_length]
	seq_out_o = r_offs[i + seq_length]
	seq_in_q = r_qlngth[i:i + seq_length]
	seq_out_q = r_qlngth[i + seq_length]
	seq_in_p = r_ptch[i:i + seq_length]
	seq_out_p = r_ptch[i + seq_length]
	dataX_o.append([offs for offs in seq_in_o])
	dataY_o.append(seq_out_o)
	dataX_q.append([qlngth for qlngth in seq_in_q])
	dataY_q.append(seq_out_q)
	dataX_p.append([ptch for ptch in seq_in_p])
	dataY_p.append(seq_out_p)
n_patterns = len(dataX_o)
print ("Total Patterns: ", n_patterns)

Total Patterns:  11884


In [ ]:
X_o = np.reshape(dataX_o, (n_patterns, seq_length, 1))
Y_o = np.reshape(dataY_o, (n_patterns, 1, 1))
X_q = np.reshape(dataX_q, (n_patterns, seq_length, 1))
Y_q = np.reshape(dataY_q, (n_patterns, 1, 1))
X_p = np.reshape(dataX_p, (n_patterns, seq_length, 1))
Y_p = np_utils.to_categorical(dataY_p)

In [ ]:

inputs_o = Input(shape=(X_o.shape[1], X_o.shape[2]), name="in1")
inputs_q = Input(shape=(X_q.shape[1], X_q.shape[2]), name="in2")
inputs_p = Input(shape=(X_p.shape[1], X_p.shape[2]), name="in3")

lstm1 = layers.LSTM(128)

lstm_o = lstm1(inputs_o)
lstm_q = lstm1(inputs_q)
lstm_p = lstm1(inputs_p)

dense_o = Dense(Y_o.shape[1], activation='sigmoid', name="out1")(lstm_o)
dense_q = Dense(Y_q.shape[1], activation='sigmoid', name="out2")(lstm_q)
dense_p = Dense(Y_p.shape[1], activation='softmax', name="out3")(lstm_p)

model = keras.Model(
    inputs= [inputs_o, inputs_q, inputs_p],
    outputs=[dense_o, dense_q, dense_p],
)

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 100, 1) <dtype: 'float32'>
(None, 100, 1) <dtype: 'float32'>
(None, 100, 1) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
(None, 85) <dtype: 'float32'>
in1 [(None, 100, 1)] float32
in2 [(None, 100, 1)] float32
in3 [(None, 100, 1)] float32
lstm (None, 100, 1) float32
out1 (None, 128) float32
out2 (None, 128) float32
out3 (None, 128) float32


[None, None, None, None, None, None, None]

In [ ]:
test_X = train_data[-1000:]
train_data = train_data[:-1000]

train_data = np.asarray(train_data)

train_X, valid_X, train_ground, valid_ground = train_test_split(train_data, train_data, test_size=0.2, random_state=13)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
    metrics=["mean_squared_error"],
)

In [ ]:
#train_X, valid_X, train_Y, valid_Y = train_test_split(X_o, Y_o, test_size=0.2, random_state=13)

history = model.fit({"in1": X_o, "in2": X_q, "in3": X_p}, {"out1": Y_o, "out2": Y_q, "out3": Y_p}, batch_size=64, epochs=2)

Epoch 1/2
186/186 [==============================] - 71s 381ms/step - loss: 3.3859 - out1_loss: 6.4559e-08 - out2_loss: 6.1604e-09 - out3_loss: 3.3859 - out1_mean_squared_error: 0.0745 - out2_mean_squared_error: 0.1987 - out3_mean_squared_error: 0.0112
Epoch 2/2
186/186 [==============================] - 71s 379ms/step - loss: 3.0469 - out1_loss: 6.4559e-08 - out2_loss: 6.1604e-09 - out3_loss: 3.0469 - out1_mean_squared_error: 0.0872 - out2_mean_squared_error: 0.1038 - out3_mean_squared_error: 0.0109


In [ ]:

test_scores = model.evaluate(valid_X, valid_Y, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
np.shape(seq_in_p)

(100, 85)

In [ ]:
train_data[11980]

[0.9946581196581197,
 0.14634146341463414,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)]

In [ ]:
dataX_p.append([arr for arr in seq_in_p])